In [3]:
from jinja2 import Template
from copy import deepcopy

In [54]:
cpp_template = Template("""
#include "Arduino.h"
#undef ISR // remove the avr-gcc ISR version
extern "C" {
#include "tpl_app_config.h"
#include "tpl_os.h" //set the OSEK ISR version.
}

void setup() {
{% for task in tasks %}
  pinMode({{ task.pin }}, OUTPUT);
{%- endfor %}
}

{% for task in tasks %}
TASK({{ task.name }}) {
  static {{ task.c_type }} nb = 0;
  digitalWrite({{ task.pin }}, nb++ & 0x01);
}
{% endfor %}
""")

In [57]:
tasks = list()
task_default = {
    "alarmtime": 5000,
    "cycletime": 1000,
    "priority": 20,
    "stacksize": 256,
    "c_type": "unsigned char"
}

cycletime=100
for pin in range(8, 14):
    task = deepcopy(task_default)
    task["name"]=f"blinkTaskPin{pin}"
    task["pin"]=pin
    task["cycletime"]=cycletime
    tasks.append(task)
    cycletime=cycletime*2

In [58]:
with open("avr-AUTOSAR-trampoline-Blink.cpp", "w") as fid:
    print(cpp_template.render(tasks=tasks), file=fid)

In [34]:
import subprocess

def make(*args):
    return subprocess.check_output(["make", *args]).decode("UTF-8")

make("format")

'clang-format-7 -i -style=LLVM avr-AUTOSAR-trampoline-Blink.cpp\n'

In [8]:
with open("avr-AUTOSAR-trampoline-Blink.elf.hex.size", "r") as fid:
    keys, values = fid.readlines()
    hex_size = dict()
    for key, value in zip(keys.split("\t"), values.split("\t")):
        key = key.strip()
        value = value.strip()
        hex_size[key]=value

In [24]:
hex_size

{'text': '0',
 'data': '8024',
 'bss': '0',
 'dec': '8024',
 'hex': '1f58',
 'filename': 'avr-AUTOSAR-trampoline-Blink.elf.hex'}

In [39]:
for task in tasks:
    task["c_type"]="long long"

In [40]:
tasks

[{'alarmtime': 5000,
  'cycletime': 10,
  'priority': 20,
  'stacksize': 256,
  'c_type': 'long long',
  'name': 'blinkTaskPin8',
  'pin': 8},
 {'alarmtime': 5000,
  'cycletime': 20,
  'priority': 20,
  'stacksize': 256,
  'c_type': 'long long',
  'name': 'blinkTaskPin9',
  'pin': 9},
 {'alarmtime': 5000,
  'cycletime': 40,
  'priority': 20,
  'stacksize': 256,
  'c_type': 'long long',
  'name': 'blinkTaskPin10',
  'pin': 10},
 {'alarmtime': 5000,
  'cycletime': 80,
  'priority': 20,
  'stacksize': 256,
  'c_type': 'long long',
  'name': 'blinkTaskPin11',
  'pin': 11},
 {'alarmtime': 5000,
  'cycletime': 160,
  'priority': 20,
  'stacksize': 256,
  'c_type': 'long long',
  'name': 'blinkTaskPin12',
  'pin': 12},
 {'alarmtime': 5000,
  'cycletime': 320,
  'priority': 20,
  'stacksize': 256,
  'c_type': 'long long',
  'name': 'blinkTaskPin13',
  'pin': 13}]

In [49]:
results = dict()

for c_type in ["char", "short", "int", "long", "long long"]:
    make("clean")
    for task in tasks:
        task["c_type"]=c_type
    with open("avr-AUTOSAR-trampoline-Blink.cpp", "w") as fid:
        print(cpp_template.render(tasks=tasks), file=fid)
    make()

    with open("avr-AUTOSAR-trampoline-Blink.elf.hex.size", "r") as fid:
        keys, values = fid.readlines()
        hex_size = dict()
        for key, value in zip(keys.split("\t"), values.split("\t")):
            key = key.strip()
            value = value.strip()
            hex_size[key]=value
        results[c_type] = hex_size["data"]

In [50]:
results

{'char': '8024',
 'short': '8072',
 'int': '8072',
 'long': '8228',
 'long long': '8420'}

In [53]:
(8072-8024)/6

8.0